#Análise somente com Emojis


Essa seção é reservada para realizar a análise de sentimentos apenas com nossa ferramenta

In [ ]:
import pandas as pd

### Declaração do modelo

In [ ]:
modelo = pd.read_csv('modelo2.csv')
emojis = set(modelo['emojis']) #conjunto de emojis do nosso modelo
modelo = modelo.set_index('emojis').to_dict('dict')['polaridade'] #Nosso modelo

## Funções de limpeza de dados


### Função que declara se um char está dentro do modelo

In [ ]:
def escopoModelo(character):
  return len(emojis.intersection(character)) > 0

### Função que limpa somente uma frase

In [ ]:
def limpaFrase(frase):
  for i in frase:
    if escopoModelo(i):
      return True
  return False

### Função que limpa o DataFrame inteiro

In [ ]:
def limpaFrame(data, coluna):
  for indice, linha in data.iterrows():
    if not limpaFrase(linha[coluna]):
      data = data.drop(indice)
  return data

### Função para limpar uma lista

In [ ]:
def limpaData(data):
  for i in data:
    if not limpaFrase(i):
      data.remove(i)
  return data

## Funções de Classificação

In [ ]:
def classificaFrase(frase):
  retorno = 0
  for i in frase:
    if escopoModelo(i):
      retorno += modelo[i]
  return int(retorno)

In [ ]:
def classifica(data):
  retorno = []
  data = limpaData(data)
  for i in data:
    if classificaFrase(i) > 0:
      retorno.append(1)
    elif classificaFrase(i) == 0:
      retorno.append(0)
    else:
      retorno.append(-1)
  return retorno

## Função de acurácia de um modelo


In [ ]:
def acuracia(verdadeiros, preditos):
    num_acertos = 0
    num_total = len(verdadeiros)

    for i in range(num_total):
      if verdadeiros[i] == preditos[i]:
        num_acertos += 1

    acuracia = (num_acertos / num_total) * 100
    return acuracia


## Usando o modelo

In [ ]:
data = pd.read_csv('dados.csv')
data = limpaFrame(data, 'Tweet')

0.0


In [ ]:
data = data.replace('positive', 1).replace('neutral', 0).replace('negative', -1)
x = data['Tweet'].values.tolist()
y = data['Sentiment'].values.tolist()
classificado = classifica(x)
print(acuracia(y, classificado))

51.75819834057685


#Análise com Vader

Essa seção é reservada para realizar a análise de sentimentos por meio da ferramenta Vader.


In [ ]:
!pip install vaderSentiment

In [ ]:
#Importando a classe
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#Instanciando
analyser = SentimentIntensityAnalyzer()

In [ ]:
#Somente para limpar os dados em inglês
emojisLimpos = []
for i in data['Tweet']:
  emojiSeparados = ''
  for j in i:
    if emojis.intersection(j):
      emojiSeparados += j
  emojisLimpos.append(emojiSeparados)

emojisLimpos = pd.DataFrame(emojisLimpos)[0]

In [ ]:
def analise(texto):
 df_analisado = analyser.polarity_scores(texto)['compound'] 
 return df_analisado

#Aplicando a função e guardando na coluna new_review
vaderAnalise = emojisLimpos.apply(analise)

In [ ]:
def polariza(score):
  if score >= 0.05:
    return 1
  elif score <= -0.05:
    return -1
  return 0
  
#Aplicando
resultadoVader = vaderAnalise.apply(polariza).values.tolist()

In [ ]:
print(acuracia(y, resultadoVader))

41.56459897273805
